# Heikin-Ashi PSAR Strategy
_Roshan Mahes_

In this tutorial, we implement the so-called _Parabolic Stop and Reverse (PSAR)_ strategy. We need to import the following modules:

In [1]:
import os
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

Let's download the Google's pricing data and plot a Japanese candlestick:

In [33]:
symbol = 'GOOG'
df = yf.download(symbol, start='2020-01-01')
df.index = df.index.strftime('%Y-%m-%d') # format dates

candles = go.Candlestick(x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close, name='candles')

fig = go.Figure(candles)
fig.layout.xaxis.type = 'category' # remove weekend days
fig.layout.xaxis.dtick = 20 # show x-axis ticker once a month

fig.layout.xaxis.rangeslider.visible = False
fig.layout.title = f'Japanese Candlestick Chart ({symbol})'
fig.layout.template = 'plotly_white'

fig.show()

[*********************100%***********************]  1 of 1 completed


## The Strategy

The _Parabolic Stop and Reverse (PSAR) indicator,_ developed by J. Wells Wilder, is used by traders to determine trend direction and potential reversals in price. It is a trend-following (lagging) indicator that uses a trailing stop and reverse method called SAR (Stop and Reverse), to identify suitable exit and entry points. The concept draws on the idea that 'time is the enemy', i.e., unless a security can continue to generate more profits over time, it should be liquidated.

The PSAR indicator appears on a chart as a series of dots, either above or below an asset's price, depending on the direction the price is moving. A dot is placed below the price when it is trending upward, and above the price when it is trending downward. There is a dot for every price bar, hence the indicator is always producing information.

The parabolic SAR is calculated almost independently for each trend in the price. When the price is in an uptrend, the SAR emerges below the price and converges upwards towards it. Similarly, on a downtrend, the SAR emerges above the price and converges downwards. At each step within a trend, the SAR is calculated one period in advance, i.e., tomorrow's SAR value is built using data available today. The general formula used for this is:

\begin{align*}
SAR_t = SAR_{t-1} + \alpha (EP - SAR_{t-1}),
\end{align*}

where $SAR_t$ is the SAR value at time $t$.

The _extreme point_ $EP$ is a record kept during each trend that represents the highest value reached by the price during the current uptrend, or lowest value during a downtrend. During each period, if a new maximum (or minimum) is observed, the EP is updated with that value.

The $\alpha$ value is the _acceleration factor._ Usually, this is initially set to a value of $0.02$. The factor is increased by $0.02$ each time a new EP is recorded. The rate will then quicken to a point where the SAR converges towards the price. To prevent it from getting too large, a maximum value for the acceleration factor is normally set to $0.20$. Generally, it is preferable in stocks to set the acceleration factor to $0.01$ so that it is not too sensitive to local decreases, whereas for commodity or currency trading the preferred value is $0.02$.

There are special cases that modify the SAR value:

- If the next period's SAR value is inside (or beyond) the current period or the previous period's price range, the SAR must be set to the closest price bound. For example, if in an upward trend, the new SAR value is calculated and if it results to be more than today's or yesterday's lowest price, it must be set equal to that lower boundary.
- If the next period's SAR value is inside (or beyond) the next period's price range, a new trend direction is then signaled. The SAR must then switch sides.
- Upon a trend switch, the first SAR value for this new trend is set to the last $EP$ recorded on the prior trend. Then, the $EP$ is reset accordingly to this period's maximum, and the acceleration factor is reset to irs initial value of $0.01$ (stocks) or $0.02$ (commodities/currencies).

In [3]:
def PSAR(df, alpha_start=0.01):
    """
    Returns the dataframe with the given PSAR indicator.
    """
    
    # initialize components
    trend = [0] * len(df)
    SAR = [0] * len(df)
    Real_SAR = [0] * len(df)
    EP = [0] * len(df)
    alpha = [0] * len(df)
    
    Real_SAR[0] = df['Open'][0]
    
    alpha[1] = alpha_start
    
    if df['Close'][1] > df['Close'][0]:
        trend[1] = 1
        SAR[1] = df['High'][0]
        EP[1] = df['High'][1]
        
    else:
        trend[1] = -1
        SAR[1] = df['Low'][0]
        EP[1] = df['Low'][1]
    
    Real_SAR[1] = SAR[1]
    
    # recursion
    for t in range(2,len(df)):
        
        SAR_new = SAR[t-1] + alpha[t-1] * (EP[t-1] - SAR[t-1])
        
        # uptrend
        if trend[t-1] > 0:
            SAR[t] = min(SAR_new, df['Low'][t-1], df['Low'][t-2])
            
            if SAR[t] > df['Low'][t]:
                SAR_new = -1
            else:
                SAR_new = trend[t-1] + 1
                
        # downtrend
        else:
            SAR[t] = max(SAR_new, df['High'][t-1], df['High'][t-2])
            
            if SAR[t] < df['High'][t]:
                SAR_new = 1
            else:
                SAR_new = trend[t-1] - 1
        
        trend[t] = SAR_new
        
        # uptrend
        if trend[t] > 0:
            if trend[t] == 1:
                EP[t] = df['High'][t]
            else:
                EP[t] = max(df['High'][t], EP[t-1])
        
        # downtrend
        else:
            if trend[t] == -1:
                EP[t] = df['Low'][t]
            else:
                EP[t] = min(df['Low'][t], EP[t-1])
        
        
        if abs(trend[t]) == 1:
            Real_SAR[t] = EP[t-1] ###
            alpha[t] = alpha_start
        else:
            Real_SAR[t] = SAR[t]
            if EP[t] == EP[t-1]:
                alpha[t] = alpha[t-1]
            else:
                alpha[t] = min(alpha[t-1] + 0.02, 0.20)
        
    # store values
    df['Real SAR'] = Real_SAR
    df['Signal'] = (df['Real SAR'] < df['Close']).apply(int).diff()

    return df
    

In [34]:
# df = heikin_ashi(df)
df = PSAR(df)

buy_dates = df.loc[df['Signal']==1].index#.strftime('%Y-%m-%d')
sell_dates = df.loc[df['Signal']==-1].index#.strftime('%Y-%m-%d')

buy_price = df.loc[df['Signal']==1]['Close']
sell_price = df.loc[df['Signal']==-1]['Close']

candles = go.Candlestick(x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close, name='candles')
psar = go.Scatter(x=df.index, y=df['Real SAR'], mode='markers', name='PSAR', line={'width': 10, 'color': 'midnightblue'})

buys = go.Scatter(x=buy_dates, y=buy_price, mode='markers', marker_size=15, marker_symbol=5, marker_color='green', name='Buy', marker_line_color='black', marker_line_width=1)
sells = go.Scatter(x=sell_dates, y=sell_price, mode='markers', marker_size=15, marker_symbol=6, marker_color='red', name='Sell', marker_line_color='black', marker_line_width=1)

fig = go.Figure(data=[candles, psar, buys, sells])

fig.layout.xaxis.type = 'category' # remove weekend days
fig.layout.xaxis.dtick = 20 # show x-axis ticker once a month

fig.layout.xaxis.rangeslider.visible = False
fig.layout.title = f'PSAR indicator ({symbol})'
fig.layout.template = 'plotly_white'
fig.show()


Instead of this traditional candlestick chart, we can also use so-called _Heikin-Ashi charts_.

## Heikin-Ashi Charts

_Heikin-Ashi_ means 'average bar' in Japanese. Heikin-Ashi charts, developed by Munehisa Homma in the 1700s, display prices that, at a glance, look similar to a traditional Japanese chart. The Heikin-Ashi technique averages price data to create a Japanese candlestick chart that filters out market noise. Instead of using the open, high, low, and close like standard candlestick charts, the Heikin-Ashi technique uses a modified formula based on two-period averages. This gives the chart a smoother appearance, making it easier to spots trends and reversals, but also obscures gaps and some price data.

The formulas are as follows:

\begin{align*}
H_{open,t} &= \frac{H_{open,t-1} + H_{close,t-1}}{2}, \\
H_{close,t} &= \frac{C_{open,t} + C_{high,t} + C_{low,t} + C_{close,t}}{4}, \\
H_{high,t} &= \max\{H_{open,t}, H_{close,t}, C_{high,t}\}, \\
H_{low,t} &= \min\{H_{open,t}, H_{close,t}, C_{low,t}\},
\end{align*}

with initial condition $H_{open, 0} = C_{open,0}$. In here, $H_{open,t}$ is the opening value in the Heikin-Ashi chart at time $t \in \mathbb{N}_0$, and $C_{open,t}$ is the opening value of the stock, which is used in the traditional Japanese candlestick chart etc.

In the following function we transform a given dataframe of stock prices to a Heikin-Ashi one.

In [35]:
def heikin_ashi(df):
    """
    Converts a dataframe according to the Heikin-Ashi.
    """
    
    df_HA = pd.DataFrame(index=df.index, columns=['Open', 'High', 'Low', 'Close'])
    
    df_HA['Open'][0] = df['Open'][0]
    df_HA['Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    
    for t in range(1,len(df)):
        df_HA.iat[t,0] = (df_HA['Open'][t-1] + df_HA['Close'][t-1]) / 2
    
    df_HA['High'] = df_HA[['Open', 'Close']].join(df['High']).max(axis=1)
    df_HA['Low'] = df_HA[['Open', 'Close']].join(df['Low']).min(axis=1)
    
    return df_HA

Let's convert the Apple's (Japanese) candlestick chart to a Heikin-Ashi chart:

In [38]:
df_HA = heikin_ashi(df)
df.index = pd.to_datetime(df.index)
df_HA.index = pd.to_datetime(df_HA.index)

candle = go.Candlestick(x=df_HA.index.strftime('%Y-%m-%d'), open=df_HA['Open'], high=df_HA['High'],
                             low=df_HA['Low'], close=df_HA['Close'])
fig = go.Figure(candle)
fig.layout.xaxis.type = 'category' # remove weekend days
fig.layout.xaxis.dtick = 20 # show x-axis ticker once a month

fig.layout.xaxis.rangeslider.visible = False
fig.layout.title = 'Heikin-Ashi Chart (AAPL)'
fig.layout.template = 'plotly_white'
fig.show()

As we can see, the Heikin-Ashi technique can be used to identify a trend more easily. Because the Heikin-Ashi technique smooths price information over two periods, it makes trends, price patterns, and reversal points easier to spot. Candles on a traditional candlestick chart frequently change from up to down, which can make them difficult to interpret. Heikin-Ashi charts typically have more consecutive colored candles, helping traders to identify past price movements easily.

The Heikin-Ashi technique reduces false trading signals in sideways and choppy markets to help traders avoid placing trades during these times. For example, instead of getting two false reversal candles before a trend commences, a trader who uses the Heikin-Ashi technique is likely only to receive the valid signal.

## Heikin-Ashi PSAR

In [39]:
df = yf.download(symbol, start='2020-01-01')
df.index = df.index.strftime('%Y-%m-%d') # format dates

df = heikin_ashi(df)
df = PSAR(df)

buy_dates = df.loc[df['Signal']==1].index#.strftime('%Y-%m-%d')
sell_dates = df.loc[df['Signal']==-1].index#.strftime('%Y-%m-%d')

buy_price = df.loc[df['Signal']==1]['Close']
sell_price = df.loc[df['Signal']==-1]['Close']

candles = go.Candlestick(x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close, name='candles')
psar = go.Scatter(x=df.index, y=df['Real SAR'], mode='markers', name='PSAR', line={'width': 10, 'color': 'midnightblue'})

buys = go.Scatter(x=buy_dates, y=buy_price, mode='markers', marker_size=15, marker_symbol=5, marker_color='green', name='Buy', marker_line_color='black', marker_line_width=1)
sells = go.Scatter(x=sell_dates, y=sell_price, mode='markers', marker_size=15, marker_symbol=6, marker_color='red', name='Sell', marker_line_color='black', marker_line_width=1)

fig = go.Figure(data=[candles, psar, buys, sells])

fig.layout.xaxis.type = 'category' # remove weekend days
fig.layout.xaxis.dtick = 20 # show x-axis ticker once a month

fig.layout.xaxis.rangeslider.visible = False
fig.layout.title = f'PSAR indicator on Heikin-Ashi data ({symbol})'
fig.layout.template = 'plotly_white'

fig.show()


[*********************100%***********************]  1 of 1 completed
